In [1]:
import tensorflow as tf
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from tensorflow.contrib.framework import nest 

from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import ops

In [2]:
a = tf.zeros([256,0])

In [4]:
sess = tf.Session()
sess.run(a)

array([], shape=(256, 0), dtype=float32)

In [4]:
arr = np.arange(24).reshape(2,3,-1)

In [9]:
a = tf.placeholder(tf.float32,[None,None,None])
def fn(inputs):
    def second_fn(inputs):
        return inputs*inputs
    def first_fn(inputs):
        return tf.reduce_mean(tf.map_fn(second_fn,inputs,dtype=tf.float32),axis=0)
    
    return tf.map_fn(first_fn,inputs,dtype=tf.float32)

b = fn(a)

In [10]:
sess = tf.Session()

In [11]:
sess.run(b,{a:arr})

array([[ 26.666666,  35.666668,  46.666668,  59.666668],
       [266.66666 , 299.66666 , 334.66666 , 371.66666 ]], dtype=float32)

In [2]:
a = tf.placeholder(tf.int32)
b = tf.placeholder(tf.float32,[5,10])
c = b[a]

In [3]:
sess = tf.Session()

In [4]:
sess.run(c,{a:1 , b:np.arange(50).reshape(5,10)})

array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19.], dtype=float32)

In [2]:
def _maybe_tensor_shape_from_tensor(shape):
    if isinstance(shape, ops.Tensor):
        print("It's tensor")
        return tensor_shape.as_shape(tensor_util.constant_value(shape))
    else:
        return shape

In [3]:
def body(time,emit):
    emit = emit.write(time,tf.ones([2,3],dtype=tf.int32)*time)
    return time +1 , emit

emit_ta = tensor_array_ops.TensorArray(tf.int32,
                                     size=0,
                                     dynamic_size=True,
                                     element_shape=tensor_shape.TensorShape([2,3]))
time = tf.constant(0,dtype=tf.int32)
_,emit = tf.while_loop(lambda t,*_:t<5 , body , loop_vars=[time,emit_ta])

In [4]:
sess = tf.Session()

In [6]:
sess.run(emit.stack())

array([[[0, 0, 0],
        [0, 0, 0]],

       [[1, 1, 1],
        [1, 1, 1]],

       [[2, 2, 2],
        [2, 2, 2]],

       [[3, 3, 3],
        [3, 3, 3]],

       [[4, 4, 4],
        [4, 4, 4]]], dtype=int32)

In [3]:
t_place = tf.placeholder(tf.int32,[])

emit_ta = nest.pack_sequence_as(0,[tensor_array_ops.TensorArray(tf.float32,
                                                                 size=0,
                                                                 dynamic_size=True,
                                                                 element_shape=tensor_shape.TensorShape([16,12]))])

time = tf.constant(0,dtype=tf.int32)

def body(time,emit):
    emit = nest.map_structure(lambda ta, em: ta.write(time,em), 
                              emit,
                              tf.ones([16,12],dtype=tf.float32)*tf.cast(time,dtype=tf.float32) )
    
    time += 1
    return time,emit

time_end , aa = tf.while_loop(lambda t,*_:t<t_place ,body ,loop_vars=[time,emit_ta])

aa = aa.stack()

sess = tf.Session()

helll = sess.run(aa,feed_dict={t_place:int(5)})

helll.shape

(5, 16, 12)

In [7]:
sess.run(emit_ta.stack(),feed_dict={t_place:int(5)})

array([], shape=(0, 16, 12), dtype=float32)

In [20]:
t_place = tf.placeholder(tf.int32,[])

emit_ta = nest.pack_sequence_as(32,[tensor_array_ops.TensorArray(tf.float32,
                                                                 size=0,
                                                                 dynamic_size=True,
                                                                 element_shape=tensor_shape.TensorShape([16,12]))])

time = tf.constant(0,dtype=tf.int32)

def body(time,emit):
    emit = nest.map_structure(lambda ta, em: ta.write(time,em), 
                              emit,
                              tf.ones([16,12],dtype=tf.float32)*tf.cast(time,dtype=tf.float32) )
    
    time += 1
    return time,emit

emit = emit_ta
for i in range(11):
    time , emit = body(time, emit)
    
aa = emit.read(5)
bb = emit.gather([emit.size()-3,emit.size()-2,emit.size()-1])
sess = tf.Session()

helll = sess.run(aa,feed_dict={t_place:int(5)})

helll.shape

(16, 12)

In [21]:
bbb = sess.run(bb)

In [22]:
bbb

array([[[ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
        [ 8.,  8.,  

In [17]:
a = [0 for i in range(10,0,-1)]
b = [i*2 for i in range(10)]

In [18]:
nest.pack_sequence_as(a,b)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [21]:
z = tf.constant(np.arange(12).reshape(3,2,2))

In [22]:
tf.argmax(z,axis=-1)

<tf.Tensor 'ArgMax_1:0' shape=(3, 2) dtype=int64>

In [10]:
t_place = tf.placeholder(tf.int32,[])

emit_ta = nest.pack_sequence_as(32,[tensor_array_ops.TensorArray(tf.float32,
                                                                 size=0,
                                                                 dynamic_size=True,
                                                                 element_shape=tensor_shape.TensorShape([16,12]))])

time = tf.constant(0,dtype=tf.int32)

def body(time,emit):
    emit = nest.map_structure(lambda ta, em: ta.write(time,em), 
                              emit,
                              tf.ones([16,12],dtype=tf.float32)*tf.cast(time,dtype=tf.float32) )
    
    time += 1
    return time,emit

emit = emit_ta
for i in range(11):
    emit = emit.write(time,tf.ones([16,12],dtype=tf.float32)*tf.cast(time,dtype=tf.float32))
    time +=1
    
aa = emit.read(5)
bb = emit.gather([emit.size()-3,emit.size()-2,emit.size()-1])
sess = tf.Session()

helll = sess.run(bb,feed_dict={t_place:int(5)})

helll.shape

(3, 16, 12)

In [12]:
bb

<tf.Tensor 'TensorArrayGatherV3_3:0' shape=(3, 16, 12) dtype=float32>

In [17]:
cc = tf.unstack(tf.shape(np.arange(64).reshape(4,8,2)))

In [20]:
cc

[<tf.Tensor 'unstack_1:0' shape=() dtype=int32>,
 <tf.Tensor 'unstack_1:1' shape=() dtype=int32>,
 <tf.Tensor 'unstack_1:2' shape=() dtype=int32>]

In [18]:
sess = tf.Session()

In [19]:
sess.run(cc)

[4, 8, 2]

In [28]:
xx = tf.placeholder(tf.float32,[None,4,8])
flat_input = nest.flatten(xx)
input_shape = array_ops.shape(flat_input[0])
inputs_got_shape = tuple(input_.get_shape().with_rank_at_least(3)
                   for input_ in flat_input)

batch_size,_ = inputs_got_shape[0].as_list()[:2]

In [33]:
inputs_got_shape[0].as_list()

[None, 4, 8]

In [26]:
sess = tf.Session()

TypeError: Fetch argument None has invalid type <class 'NoneType'>